In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.preprocessing import MinMaxScaler

In [2]:
train = pd.read_csv('./output/train.csv')    
test = pd.read_csv('./output/test.csv')    
test2 = pd.read_csv('./output/test2.csv')

for col in train.columns:
    mode = train[col].mode()[0]
    test[col].fillna(mode, inplace=True)
    test2[col].fillna(mode, inplace=True)
    train[col].fillna(mode, inplace=True)   

def within_measure(x, y):
    #  3750901.5068 ≤ 𝑥 ≤ 3770901.5068
    #  −19268905.6133 ≤ 𝑦 ≤ −19208905.6133
    if 3750901.5068 <= x and x <= 3770901.5068 and -19268905.6133 <= y and y <= -19208905.6133:
        return 1
    else:
        return 0

train = train.append(test[test.x_exit==test.x_exit])
columns=[#'hash', 'trajectory_id', 
        # 'time_entry', 'time_exit', 'vmax', 'vmin','vmean', 
    'x_entry', 'y_entry', 'first', 'last',
       #'x_req', 'y_req', 
    #'x_start', 'y_start', 'distance', 'time_start',
    #   'time_delta', 'time_total', 'city_start', 'city_now', 'city_was',
     #  'x_dir', 'y_dir', 'len',
      #   'distance_to_city', 'speed',
        #'x_exit', 'y_exit', 
        ]
x_train = train[columns]
x_test = test2[columns]

scaler = MinMaxScaler()
scaler.fit(x_train.append(x_test))
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)



y_train=pd.DataFrame()

y_train['x_exit']=train['x_exit']
y_train['y_exit']=train['y_exit']

scaler.fit(y_train)
y_train = scaler.transform(y_train)

x_train = np.reshape(x_train, x_train.shape + (1,))
x_test = np.reshape(x_test, x_test.shape + (1,))


D:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017199 entries, 0 to 202936
Data columns (total 29 columns):
hash                1017199 non-null object
trajectory_id       1017199 non-null object
time_entry          1017199 non-null int64
time_exit           1017199 non-null int64
vmax                1017199 non-null float64
vmin                1017199 non-null float64
vmean               1017199 non-null float64
x_entry             1017199 non-null float64
y_entry             1017199 non-null float64
x_exit              1017199 non-null float64
y_exit              1017199 non-null float64
first               1017199 non-null bool
last                1017199 non-null bool
x_req               1017199 non-null float64
y_req               1017199 non-null float64
x_start             1017199 non-null float64
y_start             1017199 non-null float64
distance            1017199 non-null float64
time_start          1017199 non-null int64
time_delta          1017199 non-null int64
time

In [4]:
def plotmap(ycity,title):
    fig, ax = plt.subplots()
    fig.set_figheight(10)
    fig.set_figwidth(16)

    df=test2[['x_entry','y_entry']]
    df['sqrt']=city_sqrt
    df['target']=ycity
    df=df[df.sqrt==0]
    leg=['Outside','City']
    # Plot
    for data in [df[df.target==1]]:
        ax.scatter(data.x_entry.values, data.y_entry.values,c='r',s=10, alpha=0.5,label=leg.pop(0))

    # Create a Rectangle patch
    rect = patches.Rectangle((3750901.5068,-19268905.6133),3770901.5068-3750901.5068,19268905.6133-19208905.6133,linewidth=2,edgecolor='y',facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)    

    ax.set(xlabel='x', ylabel='y',
           title=title)
    ax.legend(loc="upper left", markerscale=2, scatterpoints=1, fontsize=10)
    plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.layers import LSTM
model = Sequential()
from keras.layers import Dense


size_nn = np.shape(x_train)[1]

model.add(LSTM(200, input_shape=x_train.shape[1:]))
model.add(Dense(units=500, activation='relu', input_dim=size_nn))
model.add(Dense(units=500, activation='relu'))
model.add(Dense(units=500, activation='relu'))
model.add(Dense(units=500, activation='relu'))
#model.add(SimpleRNN( units=10, activation='tanh', 
model.add(Dense(units=2, activation='softmax'))

from keras import optimizers

#sgd = optimizers.SGD(lr=0.01, clipnorm=1.)
adam = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='mean_squared_error',
              #loss='mean_absolute_error',
              optimizer=adam,
              #metrics=['mean_squared_error'],
             )



def nn():
    # batch_size=500
    
    #trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    #testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    
    model.fit(x_train, y_train, batch_size=10000, epochs=20,verbose=True)
    
    #Prediction    
    target=list(model.predict(x_test))
    
    #print('Train Evaluation:',model.evaluate(x_train, y_train.values))

    return target
  
        

city_lgbm=pd.DataFrame()


result = nn()   
result = scaler.inverse_transform(np.array(result))
city_lgbm['x_exit']=result[:,0]
city_lgbm['y_exit']=result[:,1]



Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1017199/1017199 [==============================] - 84s 82us/step - loss: 0.0517
Epoch 2/20
1017199/1017199 [==============================] - 79s 77us/step - loss: 0.0443
Epoch 3/20
1017199/1017199 [==============================] - 78s 77us/step - loss: 0.0435
Epoch 4/20
 570000/1017199 [===============>..............] - ETA: 34s - loss: 0.0434

In [ ]:
city_lgbm[:10]

In [ ]:




city = [within_measure(x, y) for x,y in zip(city_lgbm.x_exit.values,city_lgbm.y_exit.values)]
city_sqrt = [within_measure(x, y) for x,y in zip(test2.x_entry.values,test2.y_entry.values)]
city[:10]

In [ ]:
plotmap(city,'nn')

In [ ]:






submission= pd.DataFrame()
trajectory_id = test2['trajectory_id']
submission['id']=trajectory_id 
submission['target'] = city
submission.to_csv('./output/nn/lstm.csv',index=False)    

